In [1]:
if __name__ == "__main__":
    from pyspark.sql import SparkSession

    spark = (
        SparkSession.builder.master("local")
        .appName("Word Count")
        .config("spark.some.config.option", "some-value")
        .getOrCreate()
    )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/02 16:06:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
static = spark.read.json("/data/activity-data")
dataSchema = static.schema

In [3]:
dataSchema

StructType([StructField('Arrival_Time', LongType(), True), StructField('Creation_Time', LongType(), True), StructField('Device', StringType(), True), StructField('Index', LongType(), True), StructField('Model', StringType(), True), StructField('User', StringType(), True), StructField('gt', StringType(), True), StructField('x', DoubleType(), True), StructField('y', DoubleType(), True), StructField('z', DoubleType(), True)])

In [4]:
static.show(5)

+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
| Arrival_Time|      Creation_Time|  Device|Index| Model|User|   gt|           x|           y|           z|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
|1424686735090|1424686733090638193|nexus4_1|   18|nexus4|   g|stand| 3.356934E-4|-5.645752E-4|-0.018814087|
|1424686735292|1424688581345918092|nexus4_2|   66|nexus4|   g|stand|-0.005722046| 0.029083252| 0.005569458|
|1424686735500|1424686733498505625|nexus4_1|   99|nexus4|   g|stand|   0.0078125|-0.017654419| 0.010025024|
|1424686735691|1424688581745026978|nexus4_2|  145|nexus4|   g|stand|-3.814697E-4|   0.0184021|-0.013656616|
|1424686735890|1424688581945252808|nexus4_2|  185|nexus4|   g|stand|-3.814697E-4|-0.031799316| -0.00831604|
+-------------+-------------------+--------+-----+------+----+-----+------------+------------+------------+
only showing top 5 rows



In [5]:
streaming = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1)\
    .json("/data/activity-data")

In [6]:
activityCounts = streaming.groupBy("gt").count()
activityCounts

DataFrame[gt: string, count: bigint]

In [7]:
spark.conf.set("spark.sql.shuffle.partitions", 5)

In [8]:
# 메모리 싱크

activityQuery = activityCounts.writeStream.queryName("activity_counts")\
    .format("memory").outputMode("complete")\
    .start()

24/04/02 16:07:50 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-29420ad7-4e11-4ebe-bd95-cf363ed4a0c5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 16:07:50 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [9]:
# 코드 실행시 백그라운드에서 스트리밍 연산 시작됨.
# 그런데 실행이 끝나지 않아서 주피터가 다른 셀 실행 못함 .???
# -> timeout=## 파라미터로 설정 가능

# activityQuery.awaitTermination(timeout=10)

In [10]:
from time import sleep

# 10초 후에 스트리밍 쿼리 종료
activityQuery.awaitTermination(timeout=10)

for x in range(5):
    spark.sql("SELECT * FROM activity_counts").show()
    sleep(1)

+----------+------+
|        gt| count|
+----------+------+
|       sit|418471|
|     stand|387084|
|stairsdown|318313|
|      walk|450702|
|  stairsup|355543|
|      null|355192|
|      bike|367109|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|467703|
|     stand|432616|
|stairsdown|355759|
|      walk|503722|
|  stairsup|397393|
|      null|396980|
|      bike|410297|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|529241|
|     stand|489536|
|stairsdown|402587|
|      walk|569997|
|  stairsup|449685|
|      null|449217|
|      bike|464277|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|578473|
|     stand|535076|
|stairsdown|440060|
|      walk|623016|
|  stairsup|491513|
|      null|491001|
|      bike|507457|
+----------+------+

+----------+------+
|        gt| count|
+----------+------+
|       sit|627709|
|     stand|580615|
|stairsdown|4775

In [11]:
from pyspark.sql.functions import expr

simpleTransform = streaming.withColumn("stairs", expr("gt like '%stairs%'"))\
    .where("stairs")\
    .where("gt is not null")\
    .select("gt", "model", "arrival_time", "creation_time")\
    .writeStream\
    .queryName("simple_transform")\
    .format("memory")\
    .outputMode("append")\
    .start()

24/04/02 16:08:05 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-fe0b2a40-f761-4b02-beee-a5abe1eb58fc. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 16:08:05 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [26]:
deviceModelStats = streaming.cube("gt", "model").avg()\
    .drop("avg(Arrival_time)")\
    .drop("avg(Creation_Time)")\
    .drop("avg(Index)")\
    .writeStream.queryName("device_counts").format("memory")\
    .outputMode("complete")\
    .start()

24/04/02 16:11:26 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-146a11c6-f6ec-48c2-a7b4-bd6047e48bcd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 16:11:26 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [23]:
spark.sql("SELECT * FROM device_counts").show(5)

+--------+------+--------------------+--------------------+--------------------+
|      gt| model|              avg(x)|              avg(y)|              avg(z)|
+--------+------+--------------------+--------------------+--------------------+
|     sit|  null|-5.49433244039590...|2.791446281700068E-4|-2.33994461689890...|
|    walk|nexus4|-0.00390116006094368|0.001052508689953...|-6.95435553042992...|
|    walk|  null|-0.00390116006094368|0.001052508689953...|-6.95435553042992...|
|stairsup|  null|-0.02479965287771635|-0.00800392344379...|  -0.100340884150604|
|   stand|  null|-3.11082189691724...|3.218461665975318E-4|2.141300040636475...|
+--------+------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [29]:
historicalAgg = static.groupBy("gt", "model").avg()

deviceModelStats = streaming.drop("Arrival_Time", "Creation_Time", "Index")\
    .cube("gt", "model").avg()\
    .join(historicalAgg, ["gt", "model"])\
    .writeStream.queryName("device_counts").format("memory")\
    .outputMode("complete")\
    .start()

24/04/02 16:11:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-7bd55588-6aae-4823-9309-2abf98f71db9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/02 16:11:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
